In [1]:
# Imports

import polars as pl
from rich.console import Console
from rich.table import Table
console = Console()


In [ ]:
# Import Data

dataset_path = ""
df = pl.read_parquet(dataset_path)

In [3]:
# Schema Inspection

console.print("[bold]Schema Inspection:[/bold]", style="bold")
table = Table(title="DataFrame Schema", show_header=True, header_style="bold magenta")
table.add_column("Column Name", style="cyan", no_wrap=True)
table.add_column("Data Type", style="green")

for name, dtype in df.schema.items():
    table.add_row(name, str(dtype))

console.print(table)


shape_info = f"""
[bold]DataFrame Shape:[/bold] [cyan]({df.shape[0]}, {df.shape[1]})[/cyan]
[bold]Number of packages:[/bold] [green]{df.shape[0]}[/green]
[bold]Number of columns:[/bold] [green]{df.shape[1]}[/green]
"""

console.print(shape_info)

Schema Inspection:

                DataFrame Schema                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Column Name                      ┃ Data Type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ name                             │ String    │
│ build_system                     │ String    │
│ dh_auto_configure                │ String    │
│ dh_auto_build                    │ String    │
│ dh_auto_test                     │ String    │
│ build_stderr                     │ String    │
│ build_return_code                │ Int64     │
│ test_stdout                      │ String    │
│ test_stderr                      │ String    │
│ test_returncode                  │ Int64     │
│ test_detected                    │ Int64     │
│ testing_framework                │ String    │
│ test_stdout_diff                 │ String    │
│ test_stderr_diff                 │ String    │
│ package_viable_for_test_dataset  │ Int64     │
│ rebuild_stderr                   │ String    │
│ rebuild_returncode               │ Int64     │
│ modified_rebuild_stderr          │ String    │
│ modified_rebuild_returncode      │ Int64     │
│ test_stdout_for_modified_package │ String    │
│ test_stderr_for_modified_package │ String    │
│ test_passed                      │ Int64     │
└──────────────────────────────────┴───────────┘

DataFrame Shape: (101, 22)
Number of packages: 101
Number of columns: 22

In [4]:
# Build Summary Statistics Table

console.print("[bold][cyan]Build Return Code Analysis:[/cyan][/bold]", style="bold")

return_code_mapping = {
    0: "Build Successful",
    1: "Timeout",
    2: "Build Failed",
    3: "Not Attempted",
}

build_stats = (df
            .group_by("build_return_code")
            .agg([pl.len().alias("count"),
                (pl.len() / df.shape[0] * 100).round(2).alias("percentage")])
            .sort("build_return_code"))

build_table = Table(title="Build Return Code Distribution", show_header=True, header_style="bold magenta")
build_table.add_column("Return Code", style="cyan", no_wrap=True)
build_table.add_column("Status", style="yellow")
build_table.add_column("Count", style="green", justify="right")
build_table.add_column("Percentage (%)", style="green", justify="right")

for row in build_stats.iter_rows():
    return_code, count, percentage = row
    status = return_code_mapping.get(return_code, "Unknown")
    build_table.add_row(
        str(return_code),
        status,
        f"{count}",
        f"{percentage}%"
    )

console.print(build_table)

# Failed Builds Details
console.print("[bold][cyan]Failed Builds Details for max 10 random Packages:[/cyan][/bold]", style="bold")

failed_builds_df = df.filter(pl.col("build_return_code") == 2).select([
    "name",
    "build_stderr",
])

failed_builds = failed_builds_df.sample(n=min(10, failed_builds_df.height), with_replacement=False)

for row in failed_builds.iter_rows():
    name, stderr = row
    console.print(f"[bold]Package:[/bold] [cyan]{name}[/cyan]")
    
    if stderr:
        # Show last 10 lines of stderr
        stderr_lines = stderr.strip().split('\n')
        if len(stderr_lines) > 10:
            shown_stderr = '\n'.join(stderr_lines[-10:])
            console.print(f"[bold]Build STDERR (last 10 lines):[/bold]\n{shown_stderr}")
            console.print(f"[dim]... ({len(stderr_lines) - 10} earlier lines truncated)[/dim]")
        else:
            console.print(f"[bold]Build STDERR:[/bold]\n{stderr}")
    else:
        console.print("[bold]Build STDERR:[/bold] [yellow]No error message[/yellow]")
    
    console.rule()


# Not Attempted Builds
console.print("[bold][cyan]Not Attempted Builds Details for max 10 Random packages:[/cyan][/bold]", style="bold")
not_attempted_builds_df = df.filter(pl.col("build_return_code") == 3).select([
    "name",
    "build_stderr",
])

not_attempted_builds = not_attempted_builds_df.sample(n=min(10, not_attempted_builds_df.height), with_replacement=False)

for row in not_attempted_builds.iter_rows():
    name, stderr = row
    console.print(f"[bold]Package:[/bold] [cyan]{name}[/cyan]")
    
    if stderr:
        # Show last 10 lines of stderr
        stderr_lines = stderr.strip().split('\n')
        if len(stderr_lines) > 10:
            shown_stderr = '\n'.join(stderr_lines[-10:])
            console.print(f"[bold]Build STDERR (last 10 lines):[/bold]\n{shown_stderr}")
            console.print(f"[dim]... ({len(stderr_lines) - 10} earlier lines truncated)[/dim]")
        else:
            console.print(f"[bold]Build STDERR:[/bold]\n{stderr}")
    else:
        console.print("[bold]Build STDERR:[/bold] [yellow]No error message[/yellow]")
    
    console.rule()

# Timed Out Builds

console.print("[bold][cyan]Timed Out Builds - max 10 Random packages:[/cyan][/bold]", style="bold")

timed_out_names = (df.filter(pl.col("build_return_code") == 1)
                    .select("name")
                    .sample(n=min(10, df.filter(pl.col("build_return_code") == 1).height), with_replacement=False)
                    .get_column("name")
                    .to_list())

if timed_out_names:
    timed_out_table = Table(show_header=True, header_style="bold yellow")
    timed_out_table.add_column("Package Name", style="cyan", no_wrap=True)
    
    for name in timed_out_names:
        timed_out_table.add_row(name)
    
    console.print(timed_out_table)
else:
    console.print("[green]No timed out builds found.[/green]")

Build Return Code Analysis:

              Build Return Code Distribution               
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Return Code ┃ Status           ┃ Count ┃ Percentage (%) ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ 0           │ Build Successful │    90 │         89.11% │
│ 1           │ Timeout          │     9 │          8.91% │
│ 2           │ Build Failed     │     2 │          1.98% │
└─────────────┴──────────────────┴───────┴────────────────┘

Failed Builds Details for max 10 random Packages:

Package: spglib

Build STDERR (last 10 lines):
  /worker/spglib-2.6.0/test/CMakeLists.txt:101 (FetchContent_MakeAvailable)



*** CMake configuration failed
E: pybuild pybuild:389: build: plugin pyproject failed with: exit code=1: python3.13 -m build 
--skip-dependency-check --no-isolation --wheel --outdir /worker/spglib-2.6.0/.pybuild/cpython3_3.13_spglib  
dh_auto_build: error: pybuild --build -i python{version} -p 3.13 --after-build "rm -rf {build_dir}/spglib/lib" 
returned exit code 13
make[1]: ***  Error 25
make: ***  Error 2
dpkg-buildpackage: error: debian/rules binary subprocess returned exit status 2

...

... (325 earlier lines truncated)

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Not Attempted Builds Details for max 10 Random packages:

Timed Out Builds - max 10 Random packages:

┏━━━━━━━━━━━━━━┓
┃ Package Name ┃
┡━━━━━━━━━━━━━━┩
│ tarantool    │
│ darkradiant  │
│ bochs        │
│ polyphone    │
│ xnnpack      │
│ rclone       │
│ openafs      │
│ kcmutils     │
│ bobcat       │
└──────────────┘

In [5]:
# Test Statistics

console.print("[bold][cyan]Test Dataset Eligibility Analysis:[/cyan][/bold]", style="bold")

successful_builds = df.filter(pl.col("build_return_code") == 0)

pipeline_stats = []

total_packages = df.height
pipeline_stats.append(("Total Packages", total_packages, 100.0))

successful_count = successful_builds.height
successful_pct = (successful_count / total_packages * 100)
pipeline_stats.append(("Successful Builds (build_return_code=0)", successful_count, successful_pct))


test_detected_stats = successful_builds.group_by("test_detected").agg(pl.len().alias("count"))

test_detected_0 = test_detected_stats.filter(pl.col("test_detected") == 0)
test_detected_0_count = test_detected_0.get_column("count").sum() if test_detected_0.height > 0 else 0
test_detected_0_pct = (test_detected_0_count / total_packages * 100)
pipeline_stats.append(("├─ No Tests Detected (test_detected=0)", test_detected_0_count, test_detected_0_pct))

test_detected_1 = test_detected_stats.filter(pl.col("test_detected") == 1)
test_detected_1_count = test_detected_1.get_column("count").sum() if test_detected_1.height > 0 else 0
test_detected_1_pct = (test_detected_1_count / total_packages * 100)
pipeline_stats.append(("└─ Tests Detected (test_detected=1)", test_detected_1_count, test_detected_1_pct))

viable_candidates = successful_builds.filter(pl.col("test_detected") == 1)
if viable_candidates.height > 0:
    viable_stats = viable_candidates.group_by("package_viable_for_test_dataset").agg(pl.len().alias("count"))
    
    viable_0 = viable_stats.filter(pl.col("package_viable_for_test_dataset") == 0)
    viable_0_count = viable_0.get_column("count").sum() if viable_0.height > 0 else 0
    viable_0_pct = (viable_0_count / total_packages * 100)
    pipeline_stats.append(("   ├─ Not Viable for Test Dataset", viable_0_count, viable_0_pct))
    
    viable_1 = viable_stats.filter(pl.col("package_viable_for_test_dataset") == 1)
    viable_1_count = viable_1.get_column("count").sum() if viable_1.height > 0 else 0
    viable_1_pct = (viable_1_count / total_packages * 100)
    pipeline_stats.append(("   └─ Viable for Test Dataset", viable_1_count, viable_1_pct))
else:
    pipeline_stats.append(("   └─ No packages with detected tests", 0, 0.0))

pipeline_table = Table(title="Package testing Pipeline Analysis", show_header=True, header_style="bold magenta")
pipeline_table.add_column("Stage", style="cyan", no_wrap=False, width=40)
pipeline_table.add_column("Count", style="green", justify="right")
pipeline_table.add_column("Percentage", style="green", justify="right")

for stage, count, percentage in pipeline_stats:
    pipeline_table.add_row(stage, f"{count}", f"{percentage:.2f}%")

console.print(pipeline_table)



# Test output for max 10 packages where output is not empty but test_detected = 0

console.print("\n[bold][cyan]Test Output Analysis - Packages with Output but No Tests Detected:[/cyan][/bold]", style="bold")

packages_with_output_no_tests = (df
    .filter(
        (pl.col("test_detected") == 0) & 
        (pl.col("test_stdout").is_not_null()) & 
        (pl.col("test_stdout") != "")
    )
    .select(["name", "test_stdout"])
    .sample(n=min(10, df.filter(
        (pl.col("test_detected") == 0) & 
        (pl.col("test_stdout").is_not_null()) & 
        (pl.col("test_stdout") != "")
    ).height), with_replacement=False)
)

if packages_with_output_no_tests.height > 0:
    
    for row in packages_with_output_no_tests.iter_rows():
        name, test_stdout = row
        console.print(f"\n[bold]Package:[/bold] [cyan]{name}[/cyan]")
        
        if test_stdout:
            stdout_lines = test_stdout.strip().split('\n')
            if len(stdout_lines) > 10:
                shown_stdout = '\n'.join(stdout_lines[-10:])
                console.print(f"[bold]Test STDOUT (last 10 lines):[/bold]\n{shown_stdout}")
                console.print(f"[dim]... ({len(stdout_lines) - 10} earlier lines truncated)[/dim]")
            else:
                console.print(f"[bold]Test STDOUT:[/bold]\n{test_stdout}")
        
        console.rule()
else:
    console.print("[green]No packages found with test output but test_detected=0[/green]")

# Test Output Diff for max 10 packages where output differs between runs

console.print("\n[bold][cyan]Test Output Analysis - Packages with Differing Test Outputs Between Runs:[/cyan][/bold]", style="bold")

packages_with_diff = (df
    .filter(
        (pl.col("test_detected") == 1) & 
        (pl.col("package_viable_for_test_dataset") == 0) 
    )
    .select(["name", "test_stdout_diff", "test_stderr_diff"])
    .sample(n=min(10, df.filter(
        (pl.col("test_detected") == 1) & 
        (pl.col("package_viable_for_test_dataset") == 0)
    ).height), with_replacement=False)
)

if packages_with_diff.height > 0:
    
    for row in packages_with_diff.iter_rows():
        name, stdout_diff, stderr_diff = row
        console.print(f"\n[bold]Package:[/bold] [cyan]{name}[/cyan]")
        
        if stdout_diff:
            stdout_diff_lines = stdout_diff.strip().split('\n')
            if len(stdout_diff_lines) > 10:
                shown_stdout_diff = '\n'.join(stdout_diff_lines[-10:])
                console.print(f"[bold]Test STDOUT Diff (last 10 lines):[/bold]\n{shown_stdout_diff}")
                console.print(f"[dim]... ({len(stdout_diff_lines) - 10} earlier lines truncated)[/dim]")
            else:
                console.print(f"[bold]Test STDOUT Diff:[/bold]\n{stdout_diff}")
        else:
            console.print("[bold]Test STDOUT Diff:[/bold] [yellow]No differences[/yellow]")
        
        if stderr_diff:
            stderr_diff_lines = stderr_diff.strip().split('\n')
            if len(stderr_diff_lines) > 10:
                shown_stderr_diff = '\n'.join(stderr_diff_lines[-10:])
                console.print(f"[bold]Test STDERR Diff (last 10 lines):[/bold]\n{shown_stderr_diff}")
                console.print(f"[dim]... ({len(stderr_diff_lines) - 10} earlier lines truncated)[/dim]")
            else:
                console.print(f"[bold]Test STDERR Diff:[/bold]\n{stderr_diff}")
        else:
            console.print("[bold]Test STDERR Diff:[/bold] [yellow]No differences[/yellow]")
        
        console.rule()
else:
    console.print("[green]No packages found with test output differences (test_detected=1 but not viable)[/green]")


Test Dataset Eligibility Analysis:

                Package testing Pipeline Analysis                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Stage                                    ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ Total Packages                           │   101 │    100.00% │
│ Successful Builds (build_return_code=0)  │    90 │     89.11% │
│ ├─ No Tests Detected (test_detected=0)   │    65 │     64.36% │
│ └─ Tests Detected (test_detected=1)      │    25 │     24.75% │
│    ├─ Not Viable for Test Dataset        │     6 │      5.94% │
│    └─ Viable for Test Dataset            │    19 │     18.81% │
└──────────────────────────────────────────┴───────┴────────────┘

Test Output Analysis - Packages with Output but No Tests Detected:

Package: pbsim

Test STDOUT:
Making check in src
make[1]: Entering directory '/worker/pbsim-1.0.3+git20180330.e014b1d+dfsg/src'
make[1]: Nothing to be done for 'check'.
make[1]: Leaving directory '/worker/pbsim-1.0.3+git20180330.e014b1d+dfsg/src'
make[1]: Entering directory '/worker/pbsim-1.0.3+git20180330.e014b1d+dfsg'
make[1]: Leaving directory '/worker/pbsim-1.0.3+git20180330.e014b1d+dfsg'

...

Package: monado

Test STDOUT Diff (last 10 lines):

-22: Randomness seeded to: 1678045610

+22: Randomness seeded to: 215391490

 22: ===============================================================================

 22: No tests ran

 22:

... (1729 earlier lines truncated)

Test STDERR Diff: No differences

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [21]:
# Rebuild and Modified Rebuild Analysis

console.print("\n[bold][cyan]Rebuild and Modified Rebuild Analysis - Package Behavior after Triggering Linking Phase for modified and non-modified packages [/cyan][/bold]", style="bold")


console.print("[bold][cyan]Rebuild Return Code Analysis:[/cyan][/bold]", style="bold")

rebuild_stats = (df
                .group_by("rebuild_returncode")
                .agg([pl.len().alias("count"),
                    (pl.len() / df.shape[0] * 100).round(2).alias("percentage")])
                .sort("rebuild_returncode"))


rebuild_table = Table(title="Rebuild Return Code Distribution", show_header=True, header_style="bold magenta")
rebuild_table.add_column("Rebuild Return Code", style="cyan", no_wrap=True)
rebuild_table.add_column("Count", style="green", justify="right")
rebuild_table.add_column("Percentage (%)", style="green", justify="right")

for row in rebuild_stats.iter_rows():
    return_code, count, percentage = row
    rebuild_table.add_row(
        str(return_code),
        f"{count}",
        f"{percentage}%"
    )

console.print(rebuild_table)

# Modified Rebuild Return Code Analysis

console.print("[bold][cyan]Modified Rebuild Return Code Analysis:[/cyan][/bold]", style="bold")

modified_rebuild_stats = (df
                .group_by("modified_rebuild_returncode")
                .agg([pl.len().alias("count"),
                    (pl.len() / df.shape[0] * 100).round(2).alias("percentage")])
                .sort("modified_rebuild_returncode"))

modified_rebuild_table = Table(title="Modified Rebuild Return Code Distribution", show_header=True, header_style="bold magenta")
modified_rebuild_table.add_column("Modified Rebuild Return Code", style="cyan", no_wrap=True)
modified_rebuild_table.add_column("Count", style="green", justify="right")
modified_rebuild_table.add_column("Percentage (%)", style="green", justify="right")

for row in modified_rebuild_stats.iter_rows():
    return_code, count, percentage = row
    modified_rebuild_table.add_row(
        str(return_code),
        f"{count}",
        f"{percentage}%"
    )

console.print(modified_rebuild_table)


# Failed Rebuild Details

console.print("\n[bold][cyan]Failed Rebuild Details for max 10 random Packages:[/cyan][/bold]", style="bold")

failed_rebuild_df = df.filter(pl.col("rebuild_returncode") == 2).select([
    "name",
    "rebuild_stderr",
])

if failed_rebuild_df.height > 0:
    failed_rebuilds = failed_rebuild_df.sample(n=min(10, failed_rebuild_df.height), with_replacement=False)
    
    for row in failed_rebuilds.iter_rows():
        name, stderr = row
        console.print(f"[bold]Package:[/bold] [cyan]{name}[/cyan]")
        
        if stderr:
            stderr_lines = stderr.strip().split('\n')
            if len(stderr_lines) > 10:
                shown_stderr = '\n'.join(stderr_lines[-10:])
                console.print(f"[bold]Rebuild STDERR (last 10 lines):[/bold]")
                console.print(shown_stderr, markup=False)
                console.print(f"[dim]... ({len(stderr_lines) - 10} earlier lines truncated)[/dim]")
            else:
                console.print(f"[bold]Rebuild STDERR:[/bold]")
                console.print(stderr, markup=False)
        else:
            console.print("[bold]Rebuild STDERR:[/bold] [yellow]No error message[/yellow]")
        
        console.rule()
else:
    console.print("[green]No failed rebuilds found.[/green]")

# Modified Rebuild Failed Details

console.print("\n[bold][cyan]Modified Rebuild Failed Details for max 10 random Packages:[/cyan][/bold]", style="bold")

modified_failed_rebuild_df = df.filter(pl.col("modified_rebuild_returncode") == 2).select([
    "name",
    "modified_rebuild_stderr",
])

if modified_failed_rebuild_df.height > 0:
    modified_failed_rebuilds = modified_failed_rebuild_df.sample(n=min(10, modified_failed_rebuild_df.height), with_replacement=False)
    
    for row in modified_failed_rebuilds.iter_rows():
        name, stderr = row
        console.print(f"[bold]Package:[/bold] [cyan]{name}[/cyan]")
        
        if stderr:
            stderr_lines = stderr.strip().split('\n')
            if len(stderr_lines) > 10:
                shown_stderr = '\n'.join(stderr_lines[-10:])
                console.print(f"[bold]Modified Rebuild STDERR (last 10 lines):[/bold]")
                console.print(shown_stderr, markup=False)
                console.print(f"[dim]... ({len(stderr_lines) - 10} earlier lines truncated)[/dim]")
            else:
                console.print(f"[bold]Modified Rebuild STDERR:[/bold]")
                console.print(stderr, markup=False)
        else:
            console.print("[bold]Modified Rebuild STDERR:[/bold] [yellow]No error message[/yellow]")
        
        console.rule()
else:
    console.print("[green]No failed modified rebuilds found.[/green]")

Rebuild and Modified Rebuild Analysis - Package Behavior after Triggering Linking Phase for modified and 
non-modified packages 

Rebuild Return Code Analysis:

        Rebuild Return Code Distribution        
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Rebuild Return Code ┃ Count ┃ Percentage (%) ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ 0                   │    68 │         67.33% │
│ 2                   │     6 │          5.94% │
│ 3                   │    27 │         26.73% │
└─────────────────────┴───────┴────────────────┘

Modified Rebuild Return Code Analysis:

        Modified Rebuild Return Code Distribution        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Modified Rebuild Return Code ┃ Count ┃ Percentage (%) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ 0                            │    57 │         56.44% │
│ 2                            │     8 │          7.92% │
│ 3                            │    36 │         35.64% │
└──────────────────────────────┴───────┴────────────────┘

...

Package: form

Modified Rebuild STDERR (last 10 lines):

make[3]: *** Waiting for unfinished jobs....
collect2: error: ld returned 1 exit status
make[3]: *** [Makefile:977: tform] Error 1
collect2: error: ld returned 1 exit status
make[3]: *** [Makefile:969: parform] Error 1
make[2]: *** [Makefile:411: install-recursive] Error 1
dh_auto_install: error: make -j20 install DESTDIR=/worker/form-4.3.1\+git20250227\+ds/debian/tmp 
AM_UPDATE_INFO_DIR=no returned exit code 2
make[1]: *** [debian/rules:22: override_dh_auto_install] Error 25
make: *** [debian/rules:11: binary] Error 2
dpkg-buildpackage: error: debian/rules binary subprocess returned exit status 2

... (22 earlier lines truncated)

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [25]:
# Comprehensive Build and Test Pipeline Analysis

console.print("\n[bold][cyan]Comprehensive Build and Test Pipeline Analysis:[/cyan][/bold]", style="bold")

pipeline_stats = []

total_packages = df.height
pipeline_stats.append(("Total Packages", total_packages, 100.0))

# Step 1: Original build success (return_code=0)
successful_builds = df.filter(pl.col("build_return_code") == 0)
successful_count = successful_builds.height
successful_pct = (successful_count / total_packages * 100)
pipeline_stats.append(("├─ Successful Original Builds (build_return_code=0)", successful_count, successful_pct))

if successful_builds.height > 0:
    # Step 2: Test detection
    test_detected_stats = successful_builds.group_by("test_detected").agg(pl.len().alias("count"))
    
    test_detected_0 = test_detected_stats.filter(pl.col("test_detected") == 0)
    test_detected_0_count = test_detected_0.get_column("count").sum() if test_detected_0.height > 0 else 0
    test_detected_0_pct = (test_detected_0_count / total_packages * 100)
    pipeline_stats.append(("│  ├─ No Tests Detected (test_detected=0)", test_detected_0_count, test_detected_0_pct))
    
    test_detected_1 = test_detected_stats.filter(pl.col("test_detected") == 1)
    test_detected_1_count = test_detected_1.get_column("count").sum() if test_detected_1.height > 0 else 0
    test_detected_1_pct = (test_detected_1_count / total_packages * 100)
    pipeline_stats.append(("│  └─ Tests Detected (test_detected=1)", test_detected_1_count, test_detected_1_pct))
    
    # Step 3: Package viability for test dataset (only for packages with tests)
    viable_candidates = successful_builds.filter(pl.col("test_detected") == 1)
    if viable_candidates.height > 0:
        viable_stats = viable_candidates.group_by("package_viable_for_test_dataset").agg(pl.len().alias("count"))
        
        viable_0 = viable_stats.filter(pl.col("package_viable_for_test_dataset") == 0)
        viable_0_count = viable_0.get_column("count").sum() if viable_0.height > 0 else 0
        viable_0_pct = (viable_0_count / total_packages * 100)
        pipeline_stats.append(("│     ├─ Not Viable for Test Dataset", viable_0_count, viable_0_pct))
        
        viable_1 = viable_stats.filter(pl.col("package_viable_for_test_dataset") == 1)
        viable_1_count = viable_1.get_column("count").sum() if viable_1.height > 0 else 0
        viable_1_pct = (viable_1_count / total_packages * 100)
        pipeline_stats.append(("│     └─ Viable for Test Dataset", viable_1_count, viable_1_pct))
        
        # Step 4: Rebuild success (only for viable packages)
        viable_packages = viable_candidates.filter(pl.col("package_viable_for_test_dataset") == 1)
        if viable_packages.height > 0:
            rebuild_stats = viable_packages.group_by("rebuild_returncode").agg(pl.len().alias("count"))
            
            rebuild_0 = rebuild_stats.filter(pl.col("rebuild_returncode") == 0)
            rebuild_0_count = rebuild_0.get_column("count").sum() if rebuild_0.height > 0 else 0
            rebuild_0_pct = (rebuild_0_count / total_packages * 100)
            pipeline_stats.append(("│        ├─ Successful Rebuild (rebuild_returncode=0)", rebuild_0_count, rebuild_0_pct))
            
            rebuild_non_0 = viable_packages.filter(pl.col("rebuild_returncode") != 0).height
            rebuild_non_0_pct = (rebuild_non_0 / total_packages * 100)
            pipeline_stats.append(("│        ├─ Failed/Timeout Rebuild", rebuild_non_0, rebuild_non_0_pct))
            
            # Step 5: Modified rebuild success (only for successful rebuilds)
            successful_rebuilds = viable_packages.filter(pl.col("rebuild_returncode") == 0)
            if successful_rebuilds.height > 0:
                modified_rebuild_stats = successful_rebuilds.group_by("modified_rebuild_returncode").agg(pl.len().alias("count"))
                
                modified_rebuild_0 = modified_rebuild_stats.filter(pl.col("modified_rebuild_returncode") == 0)
                modified_rebuild_0_count = modified_rebuild_0.get_column("count").sum() if modified_rebuild_0.height > 0 else 0
                modified_rebuild_0_pct = (modified_rebuild_0_count / total_packages * 100)
                pipeline_stats.append(("│        │  ├─ Successful Modified Rebuild", modified_rebuild_0_count, modified_rebuild_0_pct))
                
                modified_rebuild_non_0 = successful_rebuilds.filter(pl.col("modified_rebuild_returncode") != 0).height
                modified_rebuild_non_0_pct = (modified_rebuild_non_0 / total_packages * 100)
                pipeline_stats.append(("│        │  ├─ Failed/Timeout Modified Rebuild", modified_rebuild_non_0, modified_rebuild_non_0_pct))
                
                # Step 6: Test results (only for successful modified rebuilds)
                successful_modified_rebuilds = successful_rebuilds.filter(pl.col("modified_rebuild_returncode") == 0)
                if successful_modified_rebuilds.height > 0:
                    test_results = successful_modified_rebuilds.filter(pl.col("test_passed").is_not_null())
                    if test_results.height > 0:
                        test_passed_stats = test_results.group_by("test_passed").agg(pl.len().alias("count"))
                        
                        test_passed_1 = test_passed_stats.filter(pl.col("test_passed") == 1)
                        test_passed_1_count = test_passed_1.get_column("count").sum() if test_passed_1.height > 0 else 0
                        test_passed_1_pct = (test_passed_1_count / total_packages * 100)
                        pipeline_stats.append(("│        │  │  ├─ Tests Passed (test_passed=1)", test_passed_1_count, test_passed_1_pct))
                        
                        test_passed_0 = test_passed_stats.filter(pl.col("test_passed") == 0)
                        test_passed_0_count = test_passed_0.get_column("count").sum() if test_passed_0.height > 0 else 0
                        test_passed_0_pct = (test_passed_0_count / total_packages * 100)
                        pipeline_stats.append(("│        │  │  └─ Tests Failed (test_passed=0)", test_passed_0_count, test_passed_0_pct))
                        
                        # Final success: Complete pipeline
                        complete_success_pct = (test_passed_1_count / total_packages * 100)
                        pipeline_stats.append(("└─ COMPLETE SUCCESS: Ready for Dataset", test_passed_1_count, complete_success_pct))
                    else:
                        pipeline_stats.append(("│        │  │  └─ No test results available", 0, 0.0))
                        pipeline_stats.append(("└─ COMPLETE SUCCESS: Ready for Dataset", 0, 0.0))
                else:
                    pipeline_stats.append(("│        │  └─ No successful modified rebuilds", 0, 0.0))
                    pipeline_stats.append(("└─ COMPLETE SUCCESS: Ready for Dataset", 0, 0.0))
            else:
                pipeline_stats.append(("│        └─ No successful rebuilds", 0, 0.0))
                pipeline_stats.append(("└─ COMPLETE SUCCESS: Ready for Dataset", 0, 0.0))
        else:
            pipeline_stats.append(("│     └─ No viable packages for rebuild testing", 0, 0.0))
            pipeline_stats.append(("└─ COMPLETE SUCCESS: Ready for Dataset", 0, 0.0))
    else:
        pipeline_stats.append(("│  └─ No packages with detected tests", 0, 0.0))
        pipeline_stats.append(("└─ COMPLETE SUCCESS: Ready for Dataset", 0, 0.0))
else:
    pipeline_stats.append(("└─ No successful builds", 0, 0.0))
    pipeline_stats.append(("└─ COMPLETE SUCCESS: Ready for Dataset", 0, 0.0))

pipeline_table = Table(title="Complete Build and Test Pipeline Analysis", show_header=True, header_style="bold magenta")
pipeline_table.add_column("Pipeline Stage", style="cyan", no_wrap=False, width=50)
pipeline_table.add_column("Count", style="green", justify="right")
pipeline_table.add_column("Percentage", style="green", justify="right")

for stage, count, percentage in pipeline_stats:

    if "COMPLETE SUCCESS" in stage:
        style = "bold green"
    elif "Failed" in stage or "Timeout" in stage:
        style = "red"
    elif "Tests Passed" in stage:
        style = "green"
    elif "Tests Failed" in stage:
        style = "yellow"
    else:
        style = "white"
    
    pipeline_table.add_row(stage, f"{count}", f"{percentage:.2f}%", style=style)

console.print(pipeline_table)

console.print("\n[bold][cyan]Pipeline Summary:[/cyan][/bold]", style="bold")

final_success_count = 0
for stage, count, percentage in pipeline_stats:
    if "COMPLETE SUCCESS" in stage:
        final_success_count = count
        break

summary_info = f"""
[bold]Total Packages:[/bold] [cyan]{total_packages}[/cyan]
[bold]Successful Original Builds:[/bold] [green]{successful_count}[/green] ([green]{successful_pct:.2f}%[/green])
[bold]Complete Pipeline Success:[/bold] [bold green]{final_success_count}[/bold green] ([bold green]{(final_success_count/total_packages*100):.2f}%[/bold green])
[bold]Pipeline Efficiency:[/bold] [yellow]{(final_success_count/successful_count*100) if successful_count > 0 else 0:.2f}%[/yellow] of successful builds complete the full pipeline
"""

console.print(summary_info)

Comprehensive Build and Test Pipeline Analysis:

                 Complete Build and Test Pipeline Analysis                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Pipeline Stage                                     ┃ Count ┃ Percentage ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│ Total Packages                                     │   101 │    100.00% │
│ ├─ Successful Original Builds                      │    90 │     89.11% │
│ (build_return_code=0)                              │       │            │
│ │  ├─ No Tests Detected (test_detected=0)          │    65 │     64.36% │
│ │  └─ Tests Detected (test_detected=1)             │    25 │     24.75% │
│ │     ├─ Not Viable for Test Dataset               │     6 │      5.94% │
│ │     └─ Viable for Test Dataset                   │    19 │     18.81% │
│ │        ├─ Successful Rebuild                     │    16 │     15.84% │
│ (rebuild_returncode=0)                             │       │            │
│ │        ├─ Failed/Timeout Rebuild                 │     3 │      2.97% │
│ │        │  ├─ Successful Modified Rebuild         │    13 │     12.87% │
│ │        │  ├─ Failed/Timeout Modified Rebuild     │     3 │      2.97% │
│ │        │  │  ├─ Tests Passed (test_passed=1)     │    11 │     10.89% │
│ │        │  │  └─ Tests Failed (test_passed=0)     │     2 │      1.98% │
│ └─ COMPLETE SUCCESS: Ready for Dataset             │    11 │     10.89% │
└────────────────────────────────────────────────────┴───────┴────────────┘

Pipeline Summary:

Total Packages: 101
Successful Original Builds: 90 (89.11%)
Complete Pipeline Success: 11 (10.89%)
Pipeline Efficiency: 12.22% of successful builds complete the full pipeline